In [3]:
# JUPYTER NOTEBOOK CELL — Clean Excel, merge coords, drop columns, save CSV

import os
import re
import numpy as np
import pandas as pd

# =========================
# Paths / Settings
# =========================
FOLDER = r"C:\Users\Julian.Diaz\OneDrive - XENITH CONSULTING PTY LTD\Documents\05_Geodatabases\05_Excel"
GEOCHEM_FILE  = os.path.join(FOLDER, "NW-East_QLD_Geochem_Drilling_Samples.xlsx")
LOCATION_FILE = os.path.join(FOLDER, "NW-East_QLD_Geochem_Drilling_Samples_Loc.xlsx")

# Output folder + filenames (CSV output)
OUTDIR = r"C:\Users\Julian.Diaz\OneDrive - XENITH CONSULTING PTY LTD\Documents\05_Geodatabases\05_Excel\NW-East_Isa"
os.makedirs(OUTDIR, exist_ok=True)
OUTPUT_CSV  = os.path.join(OUTDIR, "NW-East_Isa_Geochem_Drilling_Samples_cleaned.csv")
BACKUP_CSV  = os.path.join(OUTDIR, "NW-East_Isa_Geochem_Drilling_Samples_cleaned.backup.csv")

# Sheet selection (change if needed)
GEO_SHEET = 0
LOC_SHEET = 0

# Coordinate overwrite rule
OVERWRITE_EXISTING_COORDS = False

# Columns to convert to numbers + apply negative rules
NUMERIC_COLUMNS = [
    "Au","Au1","Au2","Au3","Au4","Cu","Pb","Zn","Ag","As","Bi","Mo","Mn","Fe","Ni","Co","Cr",
    "V","Ba","Cd","Sn","Sb","Hg","Te","P","W","Zr","Ti","Mg","Th","U","Pt","Pd","S","F"
]

# Coordinate columns expected in LOCATION_FILE
COORD_COLS = ["N_GDA2020z", "E_GDA2020z", "Latitude_GDA2020", "Longitude_GDA2020"]

# Columns to drop from final output
DROP_COLS = [
    "Job_No","HasParam","HasTextParams","Duplicate_Flag","Preferred_Interval",
    "Associated_Sample_Number","Pref_Au_Assay_Field","Wet_Dry_Flag","TagCharacter",
    "Collar_ID","Sample_ID","Load_Number","Date_Updated","User_Updating","Update_Status"
]

# =========================
# Helpers
# =========================
def normalize_number_string(s: str) -> str:
    """Normalize number-like text for robust numeric parsing (preserves decimals)."""
    if s is None or (isinstance(s, float) and np.isnan(s)):
        return ""
    if not isinstance(s, str):
        s = str(s)
    s = s.strip()
    s = s.replace("%", "").replace("−", "-")
    if "," in s and "." not in s:
        s = s.replace(",", ".")
    s = re.sub(r"[^0-9.\-]+", "", s)
    if s.count(".") > 1:
        first = s.find(".")
        s = s[: first + 1] + s[first + 1:].replace(".", "")
    return s

def to_numeric(series: pd.Series) -> pd.Series:
    return pd.to_numeric(series.map(normalize_number_string), errors="coerce")

def apply_negative_rules_to_series(s: pd.Series) -> pd.Series:
    """< -99 → NaN; other negatives → abs(value)/2"""
    s = pd.to_numeric(s, errors="coerce")
    s = s.mask(s < -99, np.nan)
    s = s.mask((s < 0) & (s >= -99), abs(s) / 2)
    return s

def make_new_hole_id(df: pd.DataFrame) -> pd.DataFrame:
    """
    New_Hole_ID = Prospect_Code + '_' + Company + '_' + Sheet_Number + '_' + (Hole_ID or Sheet_Number)
    """
    for col in ["Prospect_Code", "Company", "Sheet_Number"]:
        if col not in df.columns:
            print(f"[Warn] Missing column '{col}', using blanks.")
            df[col] = ""
        df[col] = df[col].fillna("").astype(str).str.strip()

    tail = df["Hole_ID"].fillna("").astype(str).str.strip() if "Hole_ID" in df.columns else df["Sheet_Number"]
    df["New_Hole_ID"] = (
        df["Prospect_Code"] + "_" + df["Company"] + "_" + df["Sheet_Number"] + "_" + tail
    ).str.replace(r"\s+", "_", regex=True).str.replace(r"_+", "_", regex=True).str.strip("_")
    return df

def numeric_or_nan(series: pd.Series) -> pd.Series:
    return pd.to_numeric(series, errors="coerce")

def drop_columns_if_present(df: pd.DataFrame, cols: list[str]) -> tuple[pd.DataFrame, list[str]]:
    present = [c for c in cols if c in df.columns]
    missing = [c for c in cols if c not in df.columns]
    if present:
        df = df.drop(columns=present)
    return df, missing

# =========================
# 1) Load geochem Excel, build New_Hole_ID, clean numeric columns
# =========================
df_geo = pd.read_excel(GEOCHEM_FILE, sheet_name=GEO_SHEET, dtype=str)
print(f"[Info] Loaded geochem workbook → {df_geo.shape}")

df_geo = make_new_hole_id(df_geo)

missing_cols = [c for c in NUMERIC_COLUMNS if c not in df_geo.columns]
for col in NUMERIC_COLUMNS:
    if col in df_geo.columns:
        df_geo[col] = to_numeric(df_geo[col])
        df_geo[col] = apply_negative_rules_to_series(df_geo[col])
    else:
        print(f"[Warn] Geochem: column '{col}' not found; skipped.")

if missing_cols:
    print(f"[Note] Geochem missing numeric columns (skipped): {missing_cols}")

# =========================
# 2) Load location Excel, build New_Hole_ID, prep coordinates
# =========================
df_loc = pd.read_excel(LOCATION_FILE, sheet_name=LOC_SHEET, dtype=str)
df_loc.columns = [c.strip() for c in df_loc.columns]
print(f"[Info] Loaded locations workbook → {df_loc.shape}")

df_loc = make_new_hole_id(df_loc)

missing_coords = [c for c in COORD_COLS if c not in df_loc.columns]
if missing_coords:
    print(f"[Warn] Missing coordinate columns in location file: {missing_coords}")

present_coords = [c for c in COORD_COLS if c in df_loc.columns]
coords = df_loc[["New_Hole_ID"] + present_coords].copy()

if present_coords:
    # Deduplicate by New_Hole_ID, prefer rows with more filled coords
    coords["_nn"] = coords[present_coords].notna().sum(axis=1)
    coords = coords.sort_values(["New_Hole_ID", "_nn"], ascending=[True, False])
    coords = coords.drop_duplicates(subset=["New_Hole_ID"], keep="first").drop(columns="_nn")

    for c in present_coords:
        coords[c] = numeric_or_nan(coords[c])

# =========================
# 3) Merge coordinates into geochem (left join on New_Hole_ID)
# =========================
if present_coords:
    df_merged = df_geo.merge(coords, on="New_Hole_ID", how="left", suffixes=("", "_loc"))

    for c in present_coords:
        incoming = pd.to_numeric(df_merged[c], errors="coerce")
        if c in df_geo.columns:
            if OVERWRITE_EXISTING_COORDS:
                df_geo[c] = incoming
            else:
                existing = pd.to_numeric(df_geo[c], errors="coerce")
                df_geo[c] = existing.where(existing.notna(), incoming)
        else:
            df_geo[c] = incoming

# =========================
# 4) Drop requested columns
# =========================
df_geo, missing_drop = drop_columns_if_present(df_geo, DROP_COLS)
if missing_drop:
    print(f"[Note] Drop list columns not found (already absent): {missing_drop}")

# =========================
# 5) Save CSV (backup if exists)
# =========================
if os.path.exists(OUTPUT_CSV):
    if not os.path.exists(BACKUP_CSV):
        prev = pd.read_csv(OUTPUT_CSV, low_memory=False)
        prev.to_csv(BACKUP_CSV, index=False)
        print(f"[Backup created] {BACKUP_CSV}")
    else:
        print(f"[Backup exists] {BACKUP_CSV}")

df_geo.to_csv(OUTPUT_CSV, index=False)
print(f"[Saved] Cleaned and merged CSV → {OUTPUT_CSV}")

# =========================
# 6) Summary
# =========================
if present_coords:
    matched = df_geo[present_coords].notna().any(axis=1)
    print(f"[Summary] Rows with ≥1 coordinate value: {int(matched.sum())} / {len(df_geo)}")
    unmatched = ~df_geo["New_Hole_ID"].isin(coords["New_Hole_ID"])
    print(f"[Summary] Rows with New_Hole_ID not found in location file: {int(unmatched.sum())}")
else:
    print("[Summary] No coordinate columns found for merge.")


[Info] Loaded geochem workbook → (382408, 63)
[Info] Loaded locations workbook → (53960, 59)
[Saved] Cleaned and merged CSV → C:\Users\Julian.Diaz\OneDrive - XENITH CONSULTING PTY LTD\Documents\05_Geodatabases\05_Excel\NW-East_Isa\NW-East_Isa_Geochem_Drilling_Samples_cleaned.csv
[Summary] Rows with ≥1 coordinate value: 382300 / 382408
[Summary] Rows with New_Hole_ID not found in location file: 108
